In [49]:
import os
import sys

from dotenv import dotenv_values
from web3 import Web3
import requests
import json
from pprint import pprint

In [50]:
conf = dotenv_values(dotenv_path='../.env')
web3 = Web3(Web3.HTTPProvider(f"https://mainnet.infura.io/v3/{conf['infura']}"))

In [51]:
# helper functions for ABIs and reading what contract contains

def get_abi(address):
    r = requests.get(
                ("https://api.etherscan.io/api"
                    "?module=contract"
                    "&action=getabi"
                    f"&address={address}"
                    f"&apikey={conf['etherscan']}")
                    )
        
    return r.json()['result']

def contract_functions(contract):
    print(*contract.functions, sep='\n')

In [52]:
# start from fuse pool directory and implementation

directory_address = '0x835482FE0532f169024d5E9410199369aAD5C77E'
directory_implementation = '0xd662efb05e8cafe35d1558b8b5323c73e2919abd'
directory_contract = web3.eth.contract(directory_address, abi = get_abi(directory_implementation))

contract_functions(directory_contract)

_editAdminWhitelist
_editDeployerWhitelist
_setDeployerWhitelistEnforcement
adminWhitelist
bookmarkPool
deployPool
deployerWhitelist
enforceDeployerWhitelist
getAllPools
getBookmarks
getPoolsByAccount
getPublicPools
getPublicPoolsByVerification
initialize
owner
poolExists
pools
renounceOwnership
setPoolName
transferOwnership


In [53]:
# Directory contains list of all created pools, choose the one with most assets

pools = directory_contract.functions.getAllPools().call()
pool = pools[6]
pool

("Tetranode's Pool",
 '0x4702D39c499236A43654c54783c3f24830E247dC',
 '0x814b02C1ebc9164972D888495927fe1697F0Fb4c',
 12149219,
 1617221487)

In [54]:
# create the pool contract object

pool_address = pool[2]
pool_contract = web3.eth.contract(pool_address, abi=get_abi(pool_address))
contract_functions(pool_contract)

pool_implementation= pool_contract.functions.comptrollerImplementation().call()
print(pool_implementation)
pool_true = web3.eth.contract(pool_address, abi=get_abi(pool_implementation))

# find the markets that the pool contains
markets = pool_true.functions.getAllMarkets().call()
markets

_acceptAdmin
_acceptImplementation
_renounceAdminRights
_renounceFuseAdminRights
_setPendingAdmin
_setPendingImplementation
admin
adminHasRights
comptrollerImplementation
fuseAdminHasRights
pendingAdmin
pendingComptrollerImplementation
0xE16DB319d9dA7Ce40b666DD2E365a4b8B3C18217


['0x989273ec41274C4227bCB878C2c26fdd3afbE70d',
 '0x558a7A68C574D83f327E7008c63A86613Ea48B4f',
 '0x325e3257286EB040AABB1128c21a55cF82e25901',
 '0x2130528060141222F0614ff80A756A3B2A24fE59',
 '0xAD1716680024F6F9AEA57Ad28b8C4Ecd2f5670CC',
 '0xdb55B77F5E8a1a41931684Cf9e4881D24E6b6CC9',
 '0x8691927a91A032c23B895130074669f52CF6b1e7',
 '0x59Bd6774C22486D9F4FAb2D448dCe4F892a9Ae25',
 '0xF6551C22276b9Bf62FaD09f6bD6Cad0264b89789',
 '0xcA56Af76B656212d768842246bF4893b56C02ABc',
 '0x1531C1a63A169aC75A2dAAe399080745fa51dE44',
 '0x185Ab80A77D362447415a5B347D7CD86ecaCC87C',
 '0xf65155C9595F99BFC193CaFF0AAb6e2a98cf68aE',
 '0xf9f0EFFE60F56e6846505501903AD047B8011C3e',
 '0xC12b58D31b97DBd7F092db5cc69ad321a0AD747E',
 '0xF317379B10D370Fec6B8103Ef2da5007d1890def',
 '0xE33928B720799127A052B65498b322A206351441',
 '0xeb37cE0DB663A742Df93E23EA7ba78016e82BE39']

In [55]:
# choose dai market and create the contract object

dai = markets[0]
dai_contract = web3.eth.contract(dai, abi=get_abi(dai))

dai_implementation = dai_contract.functions.implementation().call()
dai_true = web3.eth.contract(dai, abi=get_abi(dai_implementation))

contract_functions(dai_true)

_acceptAdmin
_addReserves
_becomeImplementation
_reduceReserves
_renounceAdminRights
_renounceFuseAdminRights
_resignImplementation
_setAdminFee
_setComptroller
_setFuseFee
_setInterestRateModel
_setPendingAdmin
_setReserveFactor
_withdrawAdminFees
_withdrawFuseFees
accrualBlockNumber
accrueInterest
admin
adminFeeMantissa
adminHasRights
allowance
approve
balanceOf
balanceOfUnderlying
borrow
borrowBalanceCurrent
borrowBalanceStored
borrowIndex
borrowRatePerBlock
comptroller
decimals
exchangeRateCurrent
exchangeRateStored
fuseAdminHasRights
fuseFeeMantissa
getAccountSnapshot
getCash
implementation
initialize
initialize
interestRateModel
isCEther
isCToken
liquidateBorrow
mint
name
pendingAdmin
redeem
redeemUnderlying
repayBorrow
repayBorrowBehalf
reserveFactorMantissa
seize
supplyRatePerBlock
symbol
totalAdminFees
totalBorrows
totalBorrowsCurrent
totalFuseFees
totalReserves
totalSupply
transfer
transferFrom
underlying


In [56]:
# each market also has information and supplied and borrowed assets
supply = dai_true.functions.totalSupply().call()
borrows = dai_true.functions.totalBorrows().call()

# this information is fed to interest rate model to get supply/borrowRatePerBlock
supply_per_block = dai_true.functions.supplyRatePerBlock().call()
borrow_per_block = dai_true.functions.borrowRatePerBlock().call()

# supply and borrow rate per block are converted into annualized return by formula given in docs:
# https://docs.rari.capital/fuse/#calculating-the-apy-using-rate-per-block
eth_mant = 1 * 10 ** 18
days_per_year = 365
blocks_per_day = 6500

def rate_to_apy(pool_rate):
    # supply/borrow_apy = ((((supply_per_block / eth_mant) * blocks_per_day + 1) ** day_per_year) - 1) * 100
    decimal_conversion = pool_rate / eth_mant
    per_day = decimal_conversion * blocks_per_day + 1
    annualized = per_day ** days_per_year
    percentage = (annualized - 1) * 100
    return percentage

print(borrows, supply)
print(supply_per_block, borrow_per_block)
print(rate_to_apy(supply_per_block), rate_to_apy(borrow_per_block))

# Numbers match that of the UI

31757429478822447808024665 35190740579870632972433333
90465793619 128323048006
23.93251726486776 35.57021779266529


In [57]:
# initialize the interets rate contract
interest_rate_address = dai_true.functions.interestRateModel().call()
interest_rate_contract = web3.eth.contract(interest_rate_address, abi = get_abi(interest_rate_address))

# Modelling interest rates
To model how much capital can be deployed while keeping rates somewhat profitable, we need to see how supply/demandRatePerBlock are created. Knowing the inputs for interest rate calculation, we can then see how depositing or borrowing x amount from the pool changes the interest rates. 

Working with DAI market, the current market balances can be found here:

Contract: ```https://etherscan.io/address/0x989273ec41274C4227bCB878C2c26fdd3afbE70d#readProxyContract```

Implementation: ```https://etherscan.io/address/0x67e70eeb9dd170f7b4a9ef620720c9069d5e706c#code```

Each market can have a separate implementation for interest rate calculation. For DAI market in this pool it is found here:

```https://etherscan.io/address/0x67e70eeb9dd170f7b4a9ef620720c9069d5e706c#code```

As shown above, the annualized interest rate shown in UI is derived from ```supplyRatePerBlock``` and ```borrowRatePerBlock```. Calculating these is done by the interestRateModel, specifically functions ```interestRateModel.getSupplyRate()``` and ```interestRateModel.getBorrowRate()```, which are called from the market contract.


## interestRateModel.getSupplyRate()

When called from the market contract, getSupplyRate() has the following inputs.

- ```getCashPrior()```

- ```totalBorrows```

- ```totalReserves``` + ```totalFuseFees``` + ```totalAdminFees```

- ```reserveFactorMantissa``` + ```fuseFeeMantissa```+ ```adminFeeMantissa```

```getCashPrior()``` is a function in the market contract which returns how much funds there are in terms of underlying in the contract. It can be called by using ```market.getCash()```

```totalBorrows``` is a value in the contract which can be read through ```totalBorrowsCurrent```. The function has a call to ```accrueInterest```, which again calls ```finishInterestAccrual```. This means that ```totalBorrows``` returned by the function is not updated and we must calculate the small update happened since last interest paid event. Therefore calculating ```totalBorrows```:

- ```blockDelta = currentBlockNumber - accrualBlockNumber()```
- ```borrowRateMantissa = interestRateModel.getBorrowRate(getCash(), totalBorrowsCurrent(), totalReserves + totalFuseFees + totalAdminFees)```
- ```simpleInterestFactor = borrowRateMantissa * blockDelta```
- ```interestAccumulated = totalBorrowsCurrent() * simpleInterestFactor```
- ```totalBorrowsNew = totalBorrowsCurrent() + interestAccumulated```

```totalReserves``` is updated when ```totalBorrows``` is updated. Calculated as follows:
- ```reserveFactor = reserveFactorMantissa()```
- ```totalReserves = totalReserves()```
- ```totalReservesNew = interestAccumulated * reserveFactor + totalReserves```

```totalFuseFees``` is updated when ```totalBorrows``` is updated. Calculated as follows:
- ```fuseFee = fuseFeeMantissa()```
- ```totalFuseFees = totalFuseFees()```
- ```totalFuseFeesNew = interestAccumulated * fuseFee + totalFuseFees```

```totalAdminFeesNew``` is updated when ```totalBorrows``` is updated. Calculated as follows:
- ```fuseFee = reserveFactorMantissa()```
- ```totalFuseFees = totalFuseFees()```
- ```totalFuseFeesNew = interestAccumulated * fuseFee + totalFuseFees```


# PROBLEM
currently some of the new numbers are calculated wrong. They all use some mantissa (liukuluku/merkitseva numero) thing and im not yet sure how they work. Reference in compound docs ```https://github.com/compound-finance/compound-protocol/blob/master/contracts/Exponential.sol```

## interestRateModel.getBorrowRate()

``````

Important functions are:

```interestRateModel.getSupplyRate();``` which has inputs: 
```getCashPrior(), totalBorrow, totalReserves + totalFuseFees + totalAdminFees, reserveFactorMantissa + fuseFeeMantissa + adminFeeMantissa```


- ```getCashPrior``` getCash()

- ```totalBorrow``` totalBorrowsCurrent()

- ```totalReserves``` totalReserves()

- ```totalFuseFees``` totalFuseFees()

- ```totalAdminFees``` totalAdminFees()

- ```reserveFactorMantissa``` reserveFactorMantissa()

- ```fuseFeeMantissa``` fuseFeeMantissa()

- ```adminFeeMantissa``` adminFeeMantissa()

and 
```interestRateModel.getBorrowRate();```
Inputs:
```getCashPrior(), totalBorrows, totalReserves + totalFuseFees + totalAdminFees```

In [58]:
# test by pulling those values above and calling the getSupplyRate function from interest rate model
# Interest rate model is found in the market contract, DAI:
# https://etherscan.io/address/0xb579d2761470bba14018959d6dffcc681c09c04b#readContract

# query values that can be easily gotten from the contract
cash_prior = dai_true.functions.getCash().call()
total_borrows_prev = dai_true.functions.totalBorrowsCurrent().call()
total_reserves_prev = dai_true.functions.totalReserves().call()
total_fuse_fees_prev = dai_true.functions.totalFuseFees().call()
total_admin_fees_prev = dai_true.functions.totalAdminFees().call()

reserve_factor_mant = dai_true.functions.reserveFactorMantissa().call()
fuse_fee_mant = dai_true.functions.fuseFeeMantissa().call()
admin_fee_mant = dai_true.functions.adminFeeMantissa().call()

# query the accumulated interest since previous event
block_delta = web3.eth.block_number- dai_true.functions.accrualBlockNumber().call()
borrow_rate_mant = interest_rate_contract.functions.getBorrowRate(
    cash_prior,
    total_borrows_prev,
    total_reserves_prev + total_fuse_fees_prev + total_admin_fees_prev
).call()
simple_interest_factor = borrow_rate_mant * block_delta
interest_accumulated = int(total_borrows_prev * simple_interest_factor)/10**18

# add the accumulated interest to previous borrows
total_borrows_new = int(
    total_borrows_prev + interest_accumulated
)
# this is likely wrong
total_reserves_new = total_reserves_prev
total_fuse_fees_new = int(
    (interest_accumulated * fuse_fee_mant)
    / (10**18)
    + total_fuse_fees_prev
)
total_admin_fees_new = int(
    (interest_accumulated * admin_fee_mant)
    / (10**18)
    + total_admin_fees_prev
)


calculated_rate_supply = interest_rate_contract.functions.getSupplyRate(
    cash_prior,
    total_borrows_new,
    total_reserves_new + total_fuse_fees_new + total_admin_fees_new,
    reserve_factor_mant + fuse_fee_mant + admin_fee_mant
).call()

queried_rate_supply = dai_true.functions.supplyRatePerBlock().call()
own_rate_supply = getSupplyRate(
    cash_prior,
    total_borrows_new,
    total_reserves_new + total_fuse_fees_new + total_admin_fees_new,
    reserve_factor_mant + fuse_fee_mant + admin_fee_mant
)

print(f"True: {queried_rate_supply}\nCalculated: {calculated_rate_supply}\nOwn calculation: {own_rate_supply}")
print(f"True: {rate_to_apy(queried_rate_supply)}\nCalculated: {rate_to_apy(calculated_rate_supply)}\nOwn calculation: {rate_to_apy(own_rate_supply)}")

cash 8920882736616735688769976, borrow 31756496135765053298180096, reserve 136296384365615834070572, mant 100000000000000000
util 7.833214216862364e+17
kink 800000000000000000
mutl per block 148639649923
base per block 11891171993
jump 2972792998477
borrow 128323793889.62885
rate to poo 115491414500.66597
True: 90465793619
Calculated: 90465800151
Own calculation: 90466322880.95839
True: 23.93251726486776
Calculated: 23.93251918434405
Own calculation: 23.932672792352605


In [59]:
calculated_rate_borrow = interest_rate_contract.functions.getBorrowRate(
    cash_prior,
    total_borrows_new,
    total_reserves_new + total_fuse_fees_new + total_admin_fees_new
).call()

# compare calculated rate with queried rate
queried_rate_borrow = dai_true.functions.borrowRatePerBlock().call()

own_calculated_borrow = getBorrowRate(
    cash_prior,
    total_borrows_new,
    total_reserves_new + total_fuse_fees_new + total_admin_fees_new
)

print(f"True: {queried_rate_borrow}\nCalculated: {calculated_rate_borrow}\nOwn calculated: {own_calculated_borrow}")
print(f"True: {rate_to_apy(queried_rate_borrow)}\nCalculated: {rate_to_apy(calculated_rate_borrow)}\nOwn calculated: {rate_to_apy(own_calculated_borrow)}")



util 7.833214216862364e+17
kink 800000000000000000
mutl per block 148639649923
base per block 11891171993
jump 2972792998477
True: 128323048006
Calculated: 128323052414
Own calculated: 128323793889.62885
True: 35.57021779266529
Calculated: 35.570219209272835
Own calculated: 35.57045749920273


# Simulation
As calculations are close enough, we now try to replicate getSupplyRate and getBorrowRate from the interestRateModel.

## getSupplyRate()
Function returns the number of how much interest accrues per block with current pool amounts and interest rate model

### Inputs:
Inputs are defined in previous MD
- ```cash```
- ```borrows```
- ```reserves```
- ```reserveFactorMantissa```

### Function:
- ```oneMinusReserveFactor = 1e18 - reserveFactorMantissa```
- ```borrowRate = getBorrowRate(cash, borrows, reserves)```
- ```rateToPool = borrowRate * oneMinusReserveFactor / 1e18```
- ```supplyRate = utilizationRate(cash, borrows, reserves) * rateToPool / 1e18```

### getBorrowRate(cash, borrow, reserves)
- ```util = utilizationRate(cash, borrows, reserves)```
- ```if util <= kink```
    - ```borrowRate = util * multiplierPerBlock / 1e18 + baseRatePerBlock ```
- ```else```
    - ```normalRate = kink * multiplierPerBlock / 1e18 + baseRatePerBlock```
    - ```excessUtil = util - kink```
    - ```borrowRate = excessUtil * jumpMultiplierPerBlock / 1e18 + normalRate```
    
### utilizationRate(cash, borrows, reserves)
- ```if borrows = 0```
    - ```utilizationRate = 0```
- ```utilizationRate = (borrows * 1e18) / (cash + borrows - reserves)```

Next everything must be formalized and a sensitivity analysis must be done.

In [60]:
def utilizationRate(cash, borrows, reserves):
    if borrows == 0:
        return 0
    else:
        return (borrows * 1e18) / (cash + borrows - reserves)
    
def getBorrowRate(cash, borrow, reserves):
    util = utilizationRate(cash, borrows, reserves)
    print('util', util)
    kink = interest_rate_contract.functions.kink().call()
    print('kink', kink)
    multiplierPerBlock = interest_rate_contract.functions.multiplierPerBlock().call()
    print('mutl per block', multiplierPerBlock)
    baseRatePerBlock = interest_rate_contract.functions.baseRatePerBlock().call()
    print('base per block', baseRatePerBlock)
    jumpMultiplierPerBlock = interest_rate_contract.functions.jumpMultiplierPerBlock().call()
    print('jump', jumpMultiplierPerBlock)
    if util <= kink:
        return ((util * multiplierPerBlock) / 1e18) + baseRatePerBlock
    else:
        normalRate = ((kink * multiplierPerBlock) / 1e18) + baseRatePerBlock
        excessUtil = util - kink
        print('normal', normalRate)
        print('excess', excessUtil)
        return ((excessUtil * jumpMultiplierPerBlock) / 1e18) + normalRate

def getSupplyRate(cash, borrows, reserves, reserveFactorMantissa):
    print(f"cash {cash}, borrow {borrows}, reserve {reserves}, mant {reserveFactorMantissa}"
        
    )
    oneMinusReserveFactor = 1e18 - reserveFactorMantissa
    borrowRate = getBorrowRate(cash, borrows, reserves)
    print('borrow',borrowRate)
    rateToPool = borrowRate * oneMinusReserveFactor / 1e18
    print('rate to poo', rateToPool)
    return utilizationRate(cash, borrows, reserves) * rateToPool / 1e18

In [61]:
getSupplyRate()

TypeError: getSupplyRate() missing 4 required positional arguments: 'cash', 'borrows', 'reserves', and 'reserveFactorMantissa'